In [1]:
import re
import requests
import bs4
from bs4 import BeautifulSoup

## Récupération des landers

In [2]:
base = "http://germany.postcode.info/"

In [3]:
res = requests.get(base)
res.status_code

200

In [4]:
soup = BeautifulSoup(res.text, 'html.parser')

In [5]:
divs = soup.find_all("div", {'class': 'content'})

In [6]:
landers = []
for div in divs:
    temp = div.find_all("a")
    for a in temp:
        landers.append(a['href'].replace('/', ''))

In [7]:
print(landers)

['baden-wurttemberg', 'bayern', 'berlin', 'brandenburg', 'bremen', 'hamburg', 'hessen', 'mecklenburg-vorpommern', 'niedersachsen', 'nordrhein-westfalen', 'rheinland-pfalz', 'saarland', 'sachsen', 'sachsen-anhalt', 'schleswig-holstein', 'thuringen']


## Récupération de tous les codes postaux

In [8]:
postcodes = []

In [9]:
for i in range(10):
    url_plz = base + f'/p/{i}'
    res = requests.get(url_plz)
    soup = BeautifulSoup(res.text, 'html.parser')
    divs = soup.find_all("div", {'class': 'cnt'})
    for div in divs:
        temp = div.find_all("a")
        for a in temp:
            postcode = a['href'].replace('/', '')
            if len(postcode) == 5:
                postcodes.append(a['href'].replace('/', ''))

In [10]:
len(postcodes)

8312

In [234]:
with open('postcodes.list', 'w') as file:
    file.write('\n'.join(postcodes))
file.close()

## Scraper pour une page

In [11]:
res = requests.get('http://germany.postcode.info/p/55278')
res.status_code

200

In [12]:
soup = BeautifulSoup(res.text, 'html.parser')

In [13]:
def parser(souo):
    content = soup.find_all("div", {'class': 'cnt'})[0]
    in_block = False
    i_line = 0
    lines = []
    line = []
    while True:
        try:
            content = content.next_element
            if isinstance(content, bs4.element.Tag):
                if content.name == 'strong' and not in_block:
                    in_block = True
                if content.name == 'br':
                    if line != []:
                        lines.append(line)
                    line = []
                    in_block = False
                    i_line = 0
            if isinstance(content, bs4.element.NavigableString) and in_block:
                i_line += 1
                if i_line == 1: # Ort
                    line.append(content.strip())
                if i_line == 2: # Plz (5-digit postcode)
                    line.append(content.strip().replace(',', '').zfill(5))
                if i_line == 3: # Lander
                    line.append(content.strip())
                if i_line == 4: # Kreise + GPS Lat, Long
                    matched = re.findall(r'^,([^,]*)[^\d]*(\d{1,}.?\d{0,})[^\d]*(\d{1,}.?\d{0,})', content)
                    temp = [line.append(item.strip()) for item in matched[0]]          
        except:
            break
    return lines

In [14]:
lines = parser(soup)

In [15]:
print(lines)

[['Dalheim', '55278', 'Rheinland-Pfalz', 'Mainz-Bingen', '49.8267', '8.2953'], ['Dexheim', '55278', 'Rheinland-Pfalz', 'Mainz-Bingen', '49.8472', '8.3167'], ['Dolgesheim', '55278', 'Rheinland-Pfalz', 'Mainz-Bingen', '49.7944', '8.2569'], ['Eimsheim', '55278', 'Rheinland-Pfalz', 'Mainz-Bingen', '49.7958', '8.2894'], ['Friesenheim', '55278', 'Rheinland-Pfalz', 'Mainz-Bingen', '49.8364', '8.2569'], ['Hahnheim', '55278', 'Rheinland-Pfalz', 'Mainz-Bingen', '49.8608', '8.2369'], ['Köngernheim', '55278', 'Rheinland-Pfalz', 'Mainz-Bingen', '49.8458', '8.2467'], ['Ludwigshöhe', '55278', 'Rheinland-Pfalz', 'Mainz-Bingen', '49.8167', '8.3436'], ['Mommenheim', '55278', 'Rheinland-Pfalz', 'Mainz-Bingen', '49.8803', '8.265'], ['Selzen', '55278', 'Rheinland-Pfalz', 'Mainz-Bingen', '49.8606', '8.2553'], ['Uelversheim', '55278', 'Rheinland-Pfalz', 'Mainz-Bingen', '49.81', '8.2886'], ['Undenheim', '55278', 'Rheinland-Pfalz', 'Mainz-Bingen', '49.8381', '8.2189'], ['Weinolsheim', '55278', 'Rheinland-Pfalz

### Factorisation

In [16]:
def scrap_postcode(postcode):
    url = f'http://germany.postcode.info/p/{postcode}'
    res = requests.get(url)
    status_code = res.status_code 
    if status_code == 200:
        page = BeautifulSoup(res.text, 'html.parser')
        return parser(page)
    print(f'{url} cannot be accessed: error {status_code}')
    return None

In [17]:
data = scrap_postcode('55278')

In [18]:
print(data)

[['Dalheim', '55278', 'Rheinland-Pfalz', 'Mainz-Bingen', '49.8267', '8.2953'], ['Dexheim', '55278', 'Rheinland-Pfalz', 'Mainz-Bingen', '49.8472', '8.3167'], ['Dolgesheim', '55278', 'Rheinland-Pfalz', 'Mainz-Bingen', '49.7944', '8.2569'], ['Eimsheim', '55278', 'Rheinland-Pfalz', 'Mainz-Bingen', '49.7958', '8.2894'], ['Friesenheim', '55278', 'Rheinland-Pfalz', 'Mainz-Bingen', '49.8364', '8.2569'], ['Hahnheim', '55278', 'Rheinland-Pfalz', 'Mainz-Bingen', '49.8608', '8.2369'], ['Köngernheim', '55278', 'Rheinland-Pfalz', 'Mainz-Bingen', '49.8458', '8.2467'], ['Ludwigshöhe', '55278', 'Rheinland-Pfalz', 'Mainz-Bingen', '49.8167', '8.3436'], ['Mommenheim', '55278', 'Rheinland-Pfalz', 'Mainz-Bingen', '49.8803', '8.265'], ['Selzen', '55278', 'Rheinland-Pfalz', 'Mainz-Bingen', '49.8606', '8.2553'], ['Uelversheim', '55278', 'Rheinland-Pfalz', 'Mainz-Bingen', '49.81', '8.2886'], ['Undenheim', '55278', 'Rheinland-Pfalz', 'Mainz-Bingen', '49.8381', '8.2189'], ['Weinolsheim', '55278', 'Rheinland-Pfalz

In [19]:
data == lines

True

## Généralisation (scrap toutes les pages)

### Récupération de tous les postcodes

In [20]:
file = open('postcodes.list', 'r')
postcodes = [line.replace('\n', '') for line in file.readlines()]
file.close()

In [21]:
len(postcodes)

8312

### Crawl toutes les pages

3 à 4s pour scraper une page, 8312 pages à crawler, temps estimé à 9h... Le programme s'arrête prématurément avant le fin dans tous les cas, pour cause : la non gestion des timeouts lors des requests. Inutile d'aller plus loin avec cette solution naïve inefficace.

**Solution :** Passer par `scrapy`.

* https://pypi.org/project/Scrapy/
* https://docs.scrapy.org/en/latest/index.html
* https://www.youtube.com/watch?v=OJ8isyws2yw

In [22]:
postcodes = postcodes[:4] # on restreint volontairement le nombre de pages à crawler pour la démonstration
with open('output.plz', 'a') as file:
    n = len(postcodes)
    for i, postcode in enumerate(postcodes):
        print(f'postcode {postcode} beeing proceed ({i+1}/{n})')
        data = scrap_postcode(postcode)
        file.write('\n'.join(','.join(item) for item in data) + "\n")
file.close()

postcode 01067 beeing proceed (1/4)
postcode 01069 beeing proceed (2/4)
postcode 01097 beeing proceed (3/4)
postcode 01099 beeing proceed (4/4)
